In [2]:
import requests
import json
import time
import pandas as pd
from pprint import pprint

In [3]:
# 1. 楽天レシピのレシピカテゴリ一覧を取得する

res = requests.get('https://app.rakuten.co.jp/services/api/Recipe/CategoryList/20170426?applicationId=1000957752279207210')
json_data = json.loads(res.text)

parent_dict = {} # mediumカテゴリの親カテゴリの辞書

df = pd.DataFrame(columns=['category1','category2','category3','categoryId','categoryName'])

for category in json_data['result']['large']:
    df = df.append({'category1':category['categoryId'],'category2':"",'category3':"",'categoryId':category['categoryId'],'categoryName':category['categoryName']}, ignore_index=True)

for category in json_data['result']['medium']:
    df = df.append({'category1':category['parentCategoryId'],'category2':category['categoryId'],'category3':"",'categoryId':str(category['parentCategoryId'])+"-"+str(category['categoryId']),'categoryName':category['categoryName']}, ignore_index=True)
    parent_dict[str(category['categoryId'])] = category['parentCategoryId']

for category in json_data['result']['small']:
    df = df.append({'category1':parent_dict[category['parentCategoryId']],'category2':category['parentCategoryId'],'category3':category['categoryId'],'categoryId':parent_dict[category['parentCategoryId']]+"-"+str(category['parentCategoryId'])+"-"+str(category['categoryId']),'categoryName':category['categoryName']}, ignore_index=True)

In [4]:
# 2. キーワードからカテゴリを抽出する
df_keyword = df.query('categoryName.str.contains("魚")', engine='python')

In [5]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
df

,category1,category2,category3,categoryId,categoryName
0,30,,,30,人気メニュー
1,31,,,31,定番の肉料理
2,32,,,32,定番の魚料理
3,33,,,33,卵料理
4,14,,,14,ご飯もの
...,...,...,...,...,...
2156,31,720,2130,31-720-2130,ローストポーク
2157,33,721,2131,33-721-2131,味付け卵
2158,18,722,2183,18-722-2183,キャロットラペ
2159,23,723,2182,23-723-2182,ミルフィーユ鍋
